Import all

In [ ]:
pip install nibabel numpy scipy matplotlib scikit-image torch torchvision pandas

GPU Available?

In [ ]:
import torch
print(torch.__version__)
print(torch.cuda.is_available())
print(torch.cuda.get_device_name(0))

AlexNet

In [ ]:
import torch
import torch.nn as nn
import nibabel as nib
import pandas as pd
import numpy as np
import scipy.ndimage
from torch.utils.data import Dataset, DataLoader
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from tqdm import tqdm


IMG_SIZE = (128, 128, 128)
BATCH_SIZE = 1
EPOCHS = 30
LR = 1e-4
DEVICE = torch.device("cuda" if torch.cuda.is_available() else "cpu")

class Nifti3DDataset(Dataset):
    def __init__(self, df, label_encoder):
        self.df = df.reset_index(drop=True)
        self.label_encoder = label_encoder
        self.target_shape = IMG_SIZE

    def __len__(self):
        return len(self.df)

    def __getitem__(self, idx):
        path = self.df.loc[idx, "Ruta"]
        label = self.df.loc[idx, "Grupo"]

        volume = nib.load(path).get_fdata().astype(np.float32)
        zoom_factors = [t / s for t, s in zip(self.target_shape, volume.shape)]
        volume = scipy.ndimage.zoom(volume, zoom=zoom_factors, order=3)

        volume = (volume - np.mean(volume)) / (np.std(volume) + 1e-8)
        volume = np.clip(volume, -3, 3)

        volume = torch.tensor(volume).unsqueeze(0)  # [1, D, H, W]
        label = torch.tensor(self.label_encoder.transform([label])[0])
        return volume, label

#Arquitectura AlexNet
class AlexNet3D(nn.Module):
    def __init__(self, num_classes):
        super(AlexNet3D, self).__init__()
        self.features = nn.Sequential(
            nn.Conv3d(1, 64, kernel_size=11, stride=4, padding=2),
            nn.ReLU(inplace=True),
            nn.MaxPool3d(kernel_size=3, stride=2),

            nn.Conv3d(64, 192, kernel_size=5, padding=2),
            nn.ReLU(inplace=True),
            nn.MaxPool3d(kernel_size=3, stride=2),

            nn.Conv3d(192, 384, kernel_size=3, padding=1),
            nn.ReLU(inplace=True),

            nn.Conv3d(384, 256, kernel_size=3, padding=1),
            nn.ReLU(inplace=True),

            nn.Conv3d(256, 256, kernel_size=3, padding=1),
            nn.ReLU(inplace=True),
            nn.MaxPool3d(kernel_size=3, stride=2)
        )

        self.avgpool = nn.AdaptiveAvgPool3d((1, 1, 1))
        self.classifier = nn.Sequential(
            nn.Dropout(),
            nn.Linear(256, 512),
            nn.ReLU(inplace=True),
            nn.Dropout(),
            nn.Linear(512, 256),
            nn.ReLU(inplace=True),
            nn.Linear(256, num_classes)
        )

    def forward(self, x):
        x = self.features(x)
        x = self.avgpool(x)
        x = torch.flatten(x, 1)
        x = self.classifier(x)
        return x

# Cargar datos
df = pd.read_csv(r"D:/ADNI Data/BETandGroups.csv")
le = LabelEncoder()
df['labels'] = le.fit_transform(df['Grupo'])

df_train, df_val = train_test_split(df, test_size=0.25, stratify=df['labels'], random_state=42)
train_dataset = Nifti3DDataset(df_train, le)
val_dataset = Nifti3DDataset(df_val, le)

train_loader = DataLoader(train_dataset, batch_size=BATCH_SIZE, shuffle=True)
val_loader = DataLoader(val_dataset, batch_size=BATCH_SIZE)

#Entrenar modelo
model = AlexNet3D(num_classes=len(le.classes_)).to(DEVICE)
criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters(), lr=LR)

for epoch in range(EPOCHS):
    model.train()
    train_loss, train_correct = 0, 0

    for inputs, labels in tqdm(train_loader, desc=f"Epoch {epoch+1}/{EPOCHS}"):
        inputs, labels = inputs.to(DEVICE), labels.to(DEVICE)
        optimizer.zero_grad()

        outputs = model(inputs)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()

        train_loss += loss.item() * inputs.size(0)
        train_correct += (outputs.argmax(1) == labels).sum().item()

    acc_train = 100 * train_correct / len(train_dataset)
    print(f"Epoch {epoch+1} - Loss: {train_loss:.4f}, Train Acc: {acc_train:.2f}%")

    model.eval()
    val_correct = 0
    with torch.no_grad():
        for inputs, labels in val_loader:
            inputs, labels = inputs.to(DEVICE), labels.to(DEVICE)
            outputs = model(inputs)
            val_correct += (outputs.argmax(1) == labels).sum().item()
    acc_val = 100 * val_correct / len(val_dataset)
    print(f" Test Acc: {acc_val:.2f}%")

# ========= GUARDAR =========
torch.save(model.state_dict(), "alexnet3d_alzheimer.pth")
print("Modelo guardado como alexnet3d_alzheimer.pth")


AlexNet con modificación porque modelo se estanca.

In [ ]:
import os
import pandas as pd
import numpy as np
import nibabel as nib
import torch
import torch.nn as nn
import torch.nn.functional as F
import scipy.ndimage
from torch.utils.data import Dataset, DataLoader
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from tqdm import tqdm

CSV_PATH = r"D:/ADNI Data/BETandGroups.csv" 
IMG_SIZE = (128, 128, 128)
BATCH_SIZE = 4
EPOCHS = 30
LR = 1e-4
DEVICE = torch.device("cuda" if torch.cuda.is_available() else "cpu")

class Nifti3DDataset(Dataset):
    def __init__(self, df, label_encoder):
        self.df = df.reset_index(drop=True)
        self.label_encoder = label_encoder
        self.target_shape = IMG_SIZE

    def __len__(self):
        return len(self.df)

    def __getitem__(self, idx):
        path = self.df.loc[idx, "Ruta"]
        label = self.df.loc[idx, "Grupo"]
        volume = nib.load(path).get_fdata().astype(np.float32)

        zoom_factors = [t / s for t, s in zip(self.target_shape, volume.shape)]
        volume = scipy.ndimage.zoom(volume, zoom=zoom_factors, order=3)

        volume = (volume - np.mean(volume)) / (np.std(volume) + 1e-8)
        volume = np.clip(volume, -3, 3)
        volume = torch.tensor(volume).unsqueeze(0)  # [1, D, H, W]
        label = torch.tensor(self.label_encoder.transform([label])[0])
        return volume, label

# ========= MODELO =========
class AlexNet3D(nn.Module):
    def __init__(self, num_classes):
        super(AlexNet3D, self).__init__()
        self.features = nn.Sequential(
            nn.Conv3d(1, 64, kernel_size=11, stride=4, padding=2),
            nn.ReLU(inplace=True),
            nn.MaxPool3d(kernel_size=3, stride=2),

            nn.Conv3d(64, 192, kernel_size=5, padding=2),
            nn.ReLU(inplace=True),
            nn.MaxPool3d(kernel_size=3, stride=2),

            nn.Conv3d(192, 384, kernel_size=3, padding=1),
            nn.ReLU(inplace=True),

            nn.Conv3d(384, 256, kernel_size=3, padding=1),
            nn.ReLU(inplace=True),

            nn.Conv3d(256, 256, kernel_size=3, padding=1),
            nn.ReLU(inplace=True),
            nn.MaxPool3d(kernel_size=3, stride=2)
        )
        self.avgpool = nn.AdaptiveAvgPool3d((1, 1, 1))
        self.classifier = nn.Sequential(
            nn.Dropout(),
            nn.Linear(256, 512),
            nn.ReLU(inplace=True),
            nn.Dropout(),
            nn.Linear(512, 256),
            nn.ReLU(inplace=True),
            nn.Linear(256, num_classes)
        )

    def forward(self, x):
        x = self.features(x)
        x = self.avgpool(x)
        x = torch.flatten(x, 1)
        return self.classifier(x)

#Cargar datos
df = pd.read_csv(CSV_PATH)
le = LabelEncoder()
df['labels'] = le.fit_transform(df['Grupo'])
df_train, df_val = train_test_split(df, test_size=0.25, stratify=df['labels'], random_state=42)

train_dataset = Nifti3DDataset(df_train, le)
val_dataset = Nifti3DDataset(df_val, le)

train_loader = DataLoader(train_dataset, batch_size=BATCH_SIZE, shuffle=True)
val_loader = DataLoader(val_dataset, batch_size=BATCH_SIZE)

#Entrenar modelo
model = AlexNet3D(num_classes=len(le.classes_)).to(DEVICE)
criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.AdamW(model.parameters(), lr=LR)
scheduler = torch.optim.lr_scheduler.ReduceLROnPlateau(
    optimizer, mode='max', factor=0.5, patience=3, cooldown=2, verbose=True, min_lr=1e-7
)

best_val_acc = 0.0

for epoch in range(EPOCHS):
    model.train()
    train_loss, train_correct = 0, 0

    for inputs, labels in tqdm(train_loader, desc=f"Epoch {epoch+1}/{EPOCHS}"):
        inputs, labels = inputs.to(DEVICE), labels.to(DEVICE)
        optimizer.zero_grad()
        outputs = model(inputs)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()

        train_loss += loss.item() * inputs.size(0)
        train_correct += (outputs.argmax(1) == labels).sum().item()

    acc_train = 100 * train_correct / len(train_dataset)

    model.eval()
    val_correct = 0
    with torch.no_grad():
        for inputs, labels in val_loader:
            inputs, labels = inputs.to(DEVICE), labels.to(DEVICE)
            outputs = model(inputs)
            val_correct += (outputs.argmax(1) == labels).sum().item()
    acc_val = 100 * val_correct / len(val_dataset)

    # Ajuste dinámico del learning rate
    scheduler.step(acc_val)

    print(f"Epoch {epoch+1} - Loss: {train_loss:.4f}, Train Acc: {acc_train:.2f}%, Val Acc: {acc_val:.2f}%, LR: {optimizer.param_groups[0]['lr']:.1e}")

    # Guardar el mejor modelo
    if acc_val > best_val_acc:
        best_val_acc = acc_val
        torch.save(model.state_dict(), "alexnet3d_alzheimer_final.pth")
        print("Modelo mejorado guardado")

print("Entrenamiento completo")


Entrenar mas epocas

In [ ]:
import torch
import torch.nn as nn
import pandas as pd
import numpy as np
import nibabel as nib
import scipy.ndimage
from torch.utils.data import DataLoader, Dataset
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from tqdm import tqdm


CSV_PATH = r"D:/ADNI Data/BETandGroups.csv"
IMG_SIZE = (128, 128, 128)
BATCH_SIZE = 4
EXTRA_EPOCHS = 10
LR = 1e-4
DEVICE = torch.device("cuda" if torch.cuda.is_available() else "cpu")
MODEL_PATH = "alexnet3d_alzheimer_extended.pth"

class Nifti3DDataset(Dataset):
    def __init__(self, df, label_encoder):
        self.df = df.reset_index(drop=True)
        self.label_encoder = label_encoder
        self.target_shape = IMG_SIZE

    def __len__(self):
        return len(self.df)

    def __getitem__(self, idx):
        path = self.df.loc[idx, "Ruta"]
        label = self.df.loc[idx, "Grupo"]
        volume = nib.load(path).get_fdata().astype(np.float32)

        zoom_factors = [t / s for t, s in zip(self.target_shape, volume.shape)]
        volume = scipy.ndimage.zoom(volume, zoom=zoom_factors, order=3)

        volume = (volume - np.mean(volume)) / (np.std(volume) + 1e-8)
        volume = np.clip(volume, -3, 3)
        volume = torch.tensor(volume).unsqueeze(0)
        label = torch.tensor(self.label_encoder.transform([label])[0])
        return volume, label

#Architectura AlexNet3D
class AlexNet3D(nn.Module):
    def __init__(self, num_classes):
        super(AlexNet3D, self).__init__()
        self.features = nn.Sequential(
            nn.Conv3d(1, 64, kernel_size=11, stride=4, padding=2),
            nn.ReLU(inplace=True),
            nn.MaxPool3d(kernel_size=3, stride=2),
            nn.Conv3d(64, 192, kernel_size=5, padding=2),
            nn.ReLU(inplace=True),
            nn.MaxPool3d(kernel_size=3, stride=2),
            nn.Conv3d(192, 384, kernel_size=3, padding=1),
            nn.ReLU(inplace=True),
            nn.Conv3d(384, 256, kernel_size=3, padding=1),
            nn.ReLU(inplace=True),
            nn.Conv3d(256, 256, kernel_size=3, padding=1),
            nn.ReLU(inplace=True),
            nn.MaxPool3d(kernel_size=3, stride=2)
        )
        self.avgpool = nn.AdaptiveAvgPool3d((1, 1, 1))
        self.classifier = nn.Sequential(
            nn.Dropout(),
            nn.Linear(256, 512),
            nn.ReLU(inplace=True),
            nn.Dropout(),
            nn.Linear(512, 256),
            nn.ReLU(inplace=True),
            nn.Linear(256, num_classes)
        )

    def forward(self, x):
        x = self.features(x)
        x = self.avgpool(x)
        x = torch.flatten(x, 1)
        return self.classifier(x)

#Cargar datos
df = pd.read_csv(CSV_PATH)
le = LabelEncoder()
df['labels'] = le.fit_transform(df['Grupo'])
df_train, df_val = train_test_split(df, test_size=0.25, stratify=df['labels'], random_state=42)

train_dataset = Nifti3DDataset(df_train, le)
val_dataset = Nifti3DDataset(df_val, le)

train_loader = DataLoader(train_dataset, batch_size=BATCH_SIZE, shuffle=True)
val_loader = DataLoader(val_dataset, batch_size=BATCH_SIZE)

#Cargar modelo anterior
model = AlexNet3D(num_classes=len(le.classes_)).to(DEVICE)
model.load_state_dict(torch.load(MODEL_PATH, map_location=DEVICE))

criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.AdamW(model.parameters(), lr=LR)
scheduler = torch.optim.lr_scheduler.ReduceLROnPlateau(
    optimizer, mode='max', factor=0.5, patience=3, cooldown=2, verbose=True, min_lr=1e-7
)

best_val_acc = 0.0

for epoch in range(EXTRA_EPOCHS):
    model.train()
    train_loss, train_correct = 0, 0

    for inputs, labels in tqdm(train_loader, desc=f"Extra Epoch {epoch+1}/{EXTRA_EPOCHS}"):
        inputs, labels = inputs.to(DEVICE), labels.to(DEVICE)
        optimizer.zero_grad()
        outputs = model(inputs)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()

        train_loss += loss.item() * inputs.size(0)
        train_correct += (outputs.argmax(1) == labels).sum().item()

    acc_train = 100 * train_correct / len(train_dataset)

    model.eval()
    val_correct = 0
    with torch.no_grad():
        for inputs, labels in val_loader:
            inputs, labels = inputs.to(DEVICE), labels.to(DEVICE)
            outputs = model(inputs)
            val_correct += (outputs.argmax(1) == labels).sum().item()

    acc_val = 100 * val_correct / len(val_dataset)

    scheduler.step(acc_val)

    print(f"Extra Epoch {epoch+1} - Loss: {train_loss:.4f}, Train Acc: {acc_train:.2f}%, Val Acc: {acc_val:.2f}%, LR: {optimizer.param_groups[0]['lr']:.1e}")

    if acc_val > best_val_acc:
        best_val_acc = acc_val
        torch.save(model.state_dict(), "alexnet3d_alzheimer_extended2.pth")
        print("Modelo mejorado guardado")

print("Entrenamiento extendido")
